In [1]:
import time
import csv
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Upstream regulation network reconstruction

The following lines of code specify that regulation will be expored at depth 3 (`config.MAX_DEPTH = 3`), on all datasources except *mirtarbase* (`config.DATA_SOURCES = set(ds) - set(['mirtarbase'])`), on a set of 3 genes `['SCN3A', 'SCN5A', 'HEY2']`. 

The regulation graph reconstruction is called with 
```
reconstructed_network = regulation.upstream_regulation(gene_list, 
                                                       already_explored = [], 
                                                       sif_network = [], 
                                                       current_depth = 0, 
                                                       explored_reg = 0)
```

Parameters are set to default values to allow re-execution of this cell without side-effect of previous executions. 

In [2]:
start_time = time.time()

# global options
config.MAX_DEPTH = 3
config.HAS_MAX_DEPTH = True
config.FAST = True
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.UNKNOWN = True
config.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

""" removing mirtarbase """
config.DATA_SOURCES = set(ds) - set(['mirtarbase'])

""" specifiyng inputs """
gene_list = ['SCN3A', 'SCN5A', 'HEY2']
#gene_list = read_input_genes('iLiverCancer1715.target')

""" reconstruction call """
reconstructed_network = regulation.upstream_regulation(gene_list, 
                                                       already_explored = [], 
                                                       sif_network = [], 
                                                       current_depth = 0, 
                                                       explored_reg = 0)

elapsed_time = round((time.time() - start_time), 2)

print("--- Upstream regulation network in %s seconds ---" % elapsed_time)

to be explored after complex decomposition ['SCN3A', 'SCN5A', 'HEY2']
new synonmys to be explored:['EIEE62', 'FFEVF4', 'NAC3', 'Nav1.3', 'CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'CHF1', 'GRIDLOCK', 'GRL', 'HERP1', 'HESR2', 'HRT2', 'bHLHb32']
		removing suffix  protein for NOG protein
				 --> NOG
		removing suffix  mRNA for SCN3A mRNA
				 --> SCN3A
		removing suffix  protein for APP protein modified form/Aluminum complex
				 --> APP modified form/Aluminum complex
		removing suffix  complex for APP modified form/Aluminum complex
				 --> APP modified form/Aluminum
		removing suffix  modified form for APP modified form/Aluminum
				 --> APP/Aluminum
		removing suffix  mRNA for SCN5A mRNA
				 --> SCN5A

Explored 4 regulators
		removing suffix  protein for NOG protein
				 --> NOG
		removing suffix  mRNA for HEY2 mRNA
				 --> HEY2

Explored 4 regulators

Explored 4 regulators

Explored 4 regulators

Explored 4 regu

		removing suffix  protein for TNF protein
				 --> TNF
		removing suffix  protein for INS protein
				 --> INS
		removing suffix  protein for IL18 protein
				 --> IL18
		removing suffix  mRNA for IFNG mRNA
				 --> IFNG
		removing suffix  mutant form for PIK3CA gene mutant form
				 --> PIK3CA gene
		removing suffix  gene for PIK3CA gene
				 --> PIK3CA
		removing suffix  mRNA for PSEN2 mRNA
				 --> PSEN2
		removing suffix  protein for Histamine/HRH2 protein complex
				 --> Histamine/HRH2 complex
		removing suffix  complex for Histamine/HRH2 complex
				 --> Histamine/HRH2
		removing suffix  mRNA for IFNG mRNA
				 --> IFNG
		removing suffix  protein for IL12A protein/IL12B protein complex
				 --> IL12A/IL12B complex
		removing suffix  complex for IL12A/IL12B complex
				 --> IL12A/IL12B
		removing suffix  mRNA for IFNG mRNA
				 --> IFNG
		removing suffix  protein for FOXP3 protein
				 --> FOXP3
		removing suffix  mRNA for IFNG mRNA
				 --> IFNG
		removing suffix  protein for I

## Metrics and SIF export of the regulation network

In [3]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 136
Number of edges = 173


In [4]:
G_unified = util.fast_reg_network_unification(G, util.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

		merging node p21CIP1 into node CDKN1A
		merging node p21CIP1 into node CDKN1A
Nodes after synonym-based unification = 135
Edges after synonym-based unification = 173


In [5]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Node | Degree Centrality |
|------|------|
| CDKN1A | 0.552 | 
| APP | 0.09 | 
| IFNG | 0.075 | 
| NOG | 0.06 | 
| MYC/Max/MIZ-1/DNMT3A/GFI1 | 0.045 | 
| BDNF | 0.045 | 
| TAp63g (tetramer) | 0.037 | 
| SMAD2-3/SMAD4/FOXO1-3a-4/FOXG1 | 0.037 | 
| INS | 0.037 | 
| CDKN1A promoter | 0.03 | 


In [6]:
bravo_main.write_to_SIF(G_unified, 'tutorial-regulation.sif')
bravo_main.write_provenance(G_unified, 'tutorial-regulation-prov.csv')

SIF network written to tutorial-regulation.sif
Basic regulation reaction provenance written to tutorial-regulation-prov.csv


---

# Upstream signaling network reconstruction

These lines would test the regulation network reconstruction on all data sources, and stop at depth 4.  
```
config.MAX_DEPTH = 4
config.DATA_SOURCES = ['ctd', 'pid']
```

```
reconstructed_network = bravo.upstream_signaling(['SCN5A', 'SCN3A', 'HEY2'], 
        already_explored = [], 
        sif_network = [], 
        current_depth = 0, 
        explored_reg = 0)
)
```


In [7]:
start_time = time.time()

# global options
config.MAX_DEPTH = 4
config.HAS_MAX_DEPTH = True
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.FINE_GRAINED_SIGNALING_SIF = False
config.FAST = True
config.UNKNOWN = True
config.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

config.DATA_SOURCES = ds

""" reconstruction call """
reconstructed_network = signaling.upstream_signaling(['SCN5A', 'SCN3A', 'HEY2'], 
                                                     already_explored = [], 
                                                     sif_network = [], 
                                                     current_depth = 0, 
                                                     explored_reg = 0)

elapsed_time = round((time.time() - start_time), 2)
print("--- Upstream signaling network assembled in %s seconds ---" % elapsed_time)

to be explored after complex decomposition ['SCN5A', 'SCN3A', 'HEY2']
new synonmys to be explored:['CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'EIEE62', 'FFEVF4', 'NAC3', 'Nav1.3', 'CHF1', 'GRIDLOCK', 'GRL', 'HERP1', 'HESR2', 'HRT2', 'bHLHb32']
		removing suffix  complex for NOTCH3 coactivator complex
				 --> NOTCH3 coactivator
		removing suffix  complex for NOTCH4 coactivator complex
				 --> NOTCH4 coactivator

Explored 2 regulators

Explored 2 regulators

Explored 2 regulators

Explored 2 regulators

Explored 2 regulators

Explored 2 regulators

Explored 2 regulators
to be explored after complex decomposition ['NOTCH3 coactivator', 'NOTCH4 coactivator']

Explored 2 regulators
Exploring done
--- Upstream signaling network assembled in 7.71 seconds ---


In [8]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 3
Number of edges = 2


In [9]:
G_unified = util.fast_reg_network_unification(G, util.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

Nodes after synonym-based unification = 3
Edges after synonym-based unification = 2


In [10]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Node | Degree Centrality |
|------|------|
| GRL | 1.0 | 
| NOTCH4 coactivator | 0.5 | 
| NOTCH3 coactivator | 0.5 | 


In [11]:
bravo_main.write_to_SIF(G_unified, 'tutorial-signaling.sif')
bravo_main.write_provenance(G_unified, 'tutorial-signaling-prov.csv')

SIF network written to tutorial-signaling.sif
Basic regulation reaction provenance written to tutorial-signaling-prov.csv
